# Sözleşme Gücü Değişikliği

## Step 1: Change Control Process
- **State Number**: 2
- **State Code**: 10370002
- **State Name**: Contract Power Change Process Initiated - Change Control
- **Description**: This state controls the change process. If all checks are passed, it moves to the next state; otherwise, it transitions to the termination state. The checks include validating the contract term, ensuring the number of power changes is less than three, and confirming the contract is active.

#### 1a. Çalışma Öncesi Script

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    return  {}
 
    

#### 1b. İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do business/mapping here
    return {"body": {}, "params": {"assetId":str(request_data["id"])}}


#### 1c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    current_state_response = state_responses_of_machine['10370002']['businessLogicResponse']
    
    servicePointId = next((attr for attr in current_state_response['attributes'] if attr['attribute']['attributeCode'] == 'ServicePointId'), None)['value']
    contractAccountId = next((attr for attr in current_state_response['attributes'] if attr['attribute']['attributeCode'] == 'ContractAccountId'), None)['value']
    contractNumber = next((attr for attr in current_state_response['attributes'] if attr['attribute']['attributeCode'] == 'ContractNumber'), None)['value']
    
    store = {'ServicePointId' : servicePointId,
             'ContractAccountId' : contractAccountId,
             'ContractNumber': contractNumber}
    
    return store


#### 1d. Çalışma Sonrası Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
def state_handler(machine, state, request_data, context, state_responses_of_machine):


    
    cancelDate = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode'] == 'ContractCancellationDate'), None)['value']
    termType = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode'] == 'TermType'), None)['values'][0]['value']
    contractPower = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode'] == 'ContractPower'), None)['values'][0]['value']
    newContractPower = request_data['newContractPower']

    if request_data['count'] > 3:
        return {"success": True, "error": {"code": "END", "detail": "Değişim Sayısı"}}
    
    if cancelDate != 99991231000000:
        return {"success": True, "error": {"code": "END", "detail": "İptal Tarihi"}}
    if termType != 'CT':
        return {"success": True, "error": {"code": "END", "detail": "Terim Tipi"}}
    if newContractPower == contractPower:
        return {"success": True, "error": {"code": "END", "detail": "Aynı Değer"}}
    
    else:
        return {"success": True, "goto": "10370002"}
 
    
    


#### 1e. İş Mantığı

In [ ]:
"""md/search"""

## Step 2: Invoice Creation
- **State Number**: 3
- **State Code**: 10370003
- **State Name**: Invoice Creation
- **Description**: After the machine is created, it waits for the first invoice to be generated. At this stage, a specific event (from Kafka) is expected.
- **Status**: Pending

#### 2a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 2b. İstek Eşleştirme Scripti 

In [ ]:
#Yok

#### 2c. State Context Değer Tutma Scripti

In [ ]:
#Yok

#### 2d. Çalışma Sonrası Scripti

In [ ]:
#Yok

## Step 3: Contract Termination
- **State Number**: 4
- **State Code**: 10370004
- **State Name**: Contract Termination
- **Description**: This state signifies the termination of the existing dual-term contract. Relevant date attributes are updated, and then it transitions to the next state.

#### 3a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 3b. İstek Eşleştirme Scripti 

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_responses_of_machine):
    
    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
    
    data = {
        "id": request_data["id"],
        "attributes": [
            {
                "attribute": {
                    "attributeCode": "ContractCancellationDate"
                },
                "value":current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationRequestDate"
                },
                "value": current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationReason"
                },
                "value": "SubscriberRequest"  
            },
            {
                "attribute": {
                    "attributeCode": "ContractNumber"
                },
                "value": context['ContractNumber']
            }
        ],
        "authorityGroup": "string",
        "contractLockID": "string"  
    }
    
    return {"body": data}


#### 3b. İstek Eşleştirme Scripti (Excell'de benden beklenen)

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    return request_data['id']

#### 3c. State Context Değer Tutma Scripti

In [ ]:
## Yok

#### 3d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
        # do workflow here
    return {"success": True, "goto": "10370005"}



## Step 4: Closure Accrual Creation
- **State Number**: 5
- **State Code**: 10370005
- **State Name**: Closure Accrual Creation
- **Description**: A closure invoice is created for the terminated contract. The invoice creation dates and indexes are processed here.
- **Status**: Pending

#### 4a. Çalışma Öncesi Scripti

In [ ]:
# Yok

#### 4b. İstek Eşleştirme Scripti 

In [ ]:
#yok

#### 4c. State Context Değer Tutma Scripti

In [ ]:
#Yok

#### 4d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
        # do workflow here
    return {"success": True, "goto": "10370006-1"}


#### 4e. İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//accruals/calculate"""

'http://thor-stage.edas1.com/host/v1//accruals/calculate'

## Step 5: Data Preparation for New Account
- **State Number**: -
- **State Code**: -
- **State Name**: Data Preparation for New Account
- **Description**: This process involves preparing the necessary data for creating a new account. Relevant attributes are queried.
- **Status**: Pending

#### 5a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 5b. İstek Eşleştirme Scripti 

In [ ]:
# machine                     : Currently executing machine instance
# state                       : Currently executing state instance
# request_data                : Original trigger data
# context                     : Machine stored context
# state_responses_of_machine  : State responses of Machine <StateCode, {prev, businessLogic, webhook}>
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):  
    contractAccountId = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='ContractAccountId'),None)['value']
    
    return {"params": {"assetId":contractAccountId}}

#### 5c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    store = {'accountInfo' : state_responses_of_machine['10370006-1']['businessLogicResponse']}
    return store

#### 5d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do workflow here
    return {"success": True, "goto": "10370006"}


#### 5e. İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//md/search/{assetId}"""##URL is  not defined

'http://thor-stage.edas1.com/host/v1//md/search/{assetId}'

## Step 6: Create New Account
- **State Number**: 6
- **State Code**: 10370006
- **State Name**: Create New Account
- **Description**: An account is created to link with the new contract. It is defined using the same attribute values as the existing account, with the validity date set to the account creation date.
- **Status** : Pending

#### 6a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 6b. İstek Eşleştirme Scripti 

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_responses_of_machine):
    

    current_date = datetime.now().strftime('%Y%m%d%H%M%S')    #accountCreateDate = next((attr for attr in prev_state_response['attributes'] if attr['attribute']['attributeCode'] == 'CreateDate'),None)['value']

    deletelist = ['AssetIdentifier']
    


    copied_json = {}
    stack = [(copied_json, state_responses_of_machine['10370006-1']['businessLogicResponse'])]

    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    for attr in copied_json['attributes']:
        if attr['attribute']['attributeCode']=='ExpiryDate':
            attr['value'] = current_date

    copied_json['attributes'] = [attr for attr in copied_json['attributes'] if attr['attribute']['attributeCode'] not in deletelist]

    
    
    return copied_json


#### 6c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    newAccountId = state_responses_of_machine['10340007']['businessLogicResponse']['id'] 
    store = {'NewAccountId' : newAccountId}
    return store


#### 6d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # do workflow here
    return {"success": True, "goto": "10370007"}


#### 6e. İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//account/create"""


'http://thor-stage.edas1.com/host/v1//account/create'

## Step 7: Creation of New Contract
- **State Number**: 7
- **State Code**: 10370007
- **State Name**: Creation of New Contract
- **Description**: A new contract is created with the new contract power information. The same attributes as the existing contract are used to create a double term contract.

#### 7a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 7b. İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_responses_of_machine):
    attribute_codes = ['ServicePointId', 'BusinessPartnerId', 'AccountId']
    json_parts = []

    for code in attribute_codes:
        value = None  # Default value
        
        for attr in request_data['attributes']:
            if attr['attribute']['attributeCode'] == code:
                value = attr['value']
                if value is None and attr['values']:
                    value = attr['values'][0]['value']  # Get value from values if available
                break
        
        json_parts.append(f'"{code}": {json.dumps(value)}')  # Create JSON key-value pairs
    
    json_output = '{' + ', '.join(json_parts) + '}'
    return json_output  # Return the JSON 



##### 7b. İstek Eşleştirme Scripti- Olması gereken(!1)

In [1]:
from datetime import datetime
import json
def state_handler(machine, state, request_data, context, state_responses_of_machine):
    # Not recursive 
    current_date = datetime.now().strftime('%Y%m%d%H%M%S') 
    newContractPower  = request_data['NewContractPower']
    
    
    changes_dict = {'ContractPower':newContractPower,
                     'ContractAccountId': context['accountInfo']['id'] }
    
    copied_json = {}
    stack = [(copied_json, request_data)]
    
    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    # Set dates for all values in copied_json
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    if item.get('value') is None and 'values' in item and item['values']:
                        item['values'][0]['startDate'] = current_date
                        item['values'][0]['endDate'] = 99991231235959
                    elif item.get('value') is not None:
                        item['startDate'] = current_date
                        item['endDate'] = 99991231235959
        elif isinstance(value, dict):
            if value.get('value') is None and 'values' in value and value['values']:
                value['values'][0]['startDate'] = current_date
                value['values'][0]['endDate'] = 99991231235959
            elif value.get('value') is not None:
                value['startDate'] = current_date
                value['endDate'] = 99991231235959

    # Update attribute values in context according to the dict above
    for code, new_value in changes_dict.items():
        for attr in copied_json.get('attributes', []):
            if attr['attribute']['attributeCode'] == code:
                if attr['value'] is None:
                    attr['values'][0]['value'] = new_value
                    attr['values'][0]['valueDesc'] = None
                else:
                    attr['valueDesc'] = None
                    attr['value'] = new_value

    return {"body":copied_json}  

#### 7c. State Context Değer Tutma Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_responses_of_machine):
    current_state_response = state_responses_of_machine
    ContractId = current_state_response['id']

    store = {'NewContractId':ContractId}
    return store

##### 7d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, state_responses_of_machine):
        # do workflow here
    return {"success": True, "goto": "END"}


#### 7e. İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//contract/create"""

'http://thor-stage.edas1.com/host/v1//contract/create'